# 1) Импорт библиотек для работы с TDA (ripser, gudhi, dionis) и классификаторами 

In [ ]:
!pip install ripser
!conda install -c conda-forge gudhi --y

In [1]:
from sklearn import metrics
import numpy as np
import re
import cython
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from ripser import Rips
from persim import PersImage
from sklearn.svm import SVC
import numpy as np
import os
import re
import pandas as pd
pd_data = {}
data_dir = '../fcla5study/connectivity_matrices/'
data_dir_targ = ''
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import numpy as np
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
#import gudhi
import persim #для построения диаграмм 

import h5py                         
import pandas as pd      

import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold
from tempfile                import mkdtemp
from shutil                  import rmtree
from sklearn.preprocessing   import LabelEncoder
from sklearn.svm             import SVC
from sklearn.ensemble        import RandomForestClassifier
from sklearn.neighbors       import KNeighborsClassifier
from sklearn.pipeline        import Pipeline
from sklearn.model_selection import GridSearchCV

import sys
try:
    import pandas as pd
except ImportError as e:
    print("pandas is required for this example. Please install with conda or pip  and then try again.")
    sys.exit()

import numpy as np
import sklearn
from sklearn import ensemble
import kmapper as km                      # для визуализации графа
#from kmapper.plotlyviz import *

import warnings
warnings.filterwarnings("ignore")
#import plotly.graph_objs as go
#from ipywidgets import (HBox, VBox)


#from sklearn_tda import sklearn_tda as tda

import matplotlib.pyplot as plt
import numpy as np
from sklearn.kernel_approximation import RBFSampler

import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage
import PIL

from persim import plot_diagrams
from ripser import ripser, lower_star_img
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from utils import generate, visualization, load
from archi import perslay, baseModel
from preprocessing import preprocess
from expe import single_run
import gudhi
from sklearn.cluster import KMeans

In [2]:
data_dir_new = '../server/fmriprep_connectivity_matrices/'
data1 = pd.read_csv('targets 22.03.2018.csv' ,sep = ';')
data1["help"] = data1["D/C"].apply(lambda x: 0 if x == "D" else 1 if x =='C' else 2)
dic = pd.Series(data1.help.values,index=data1.patient_number).to_dict() #для 100 пациентов

# Подгрузка самой диаграммы

In [ ]:
import csv

def all_diagr(dim, thresh, data_dir,hint, disease_name, disease, name_stat, name_features, name_keys, begin, end, step):
    #---------------------------------------------------------------------------------------------
    #data_dir - directory of data
    #disease - label of disease (1- SCH, 2-Bipolar, 3 - ADHD)
    #name_stat file recording all statistics of features
    #name_features - file recording features for the future classification by Katia
    #name_keys - file of labels (keys)
    #begin - start value of the radious of rips complex
    #end - final value of the radious of rips complex
    #step - step of value of the radious of rips complex
    #---------------------------------------------------------------------------------------------
    #initialization of methods for features
    labels = []   
    fs = os.listdir(data_dir)
    fs.sort()
    import re
    corr_dict = {}
    subs = [] #массив номеров поциентов
    diagrams_h1 = []
    diagrams_health = []
    diagrams_disease = []
    i = 0
    diagrams_h1 = [] #массив для персистетных диаграмм всех пациентов
    with open(name_stat, 'w') as file:
        for ind in np.arange(begin, end, step):
            rips = Rips(maxdim=2, coeff=2, do_cocycles=False,thresh=ind);
            for f in fs:
                
                if (f == '.ipynb_checkpoints' or f == '.DS_Store'):
                    continue
                sub = f[4:8]
                #print('sub', sub)
                #sub = re.findall(r'\b\d+\b',f)[0] #выделяем в имени файла пациента его порядковый номер
                subs.append(sub) 
                test = dic.get(sub) #находим метку болезни по номеру пациента из словаря
                #print(test)
                if sub not in corr_dict: #если пациента еще нет в словаре
                    #print(sub)
                  
                    if (test == 0 or test == disease):  #смотрим, относится ли пациент к классу здоровых или выбранной категории больных
                        
                        corr_dict[sub] = {} #заводим словарь {номер пациента - корреляционная матрица}
                        #corr_dict[sub][test] = np.loadtxt(data_dir+f, delimiter=',', skiprows=4)
                        corr_dict[sub][test] = pd.read_csv(data_dir+f, index_col=[0]).values #запись в словарь корелляционной матрицы
                        corr_dict[sub][test][np.abs(corr_dict[sub][test])< thresh] = 0
                        #corr_dict[sub][test][corr_dict[sub][test]< thresh] = 0
                        #corr_dict[sub][test] = corr_dict[sub][test].transpose()
    for i, sub in enumerate(corr_dict):
        dist = np.asanyarray(list(corr_dict[sub].values()))[0]
        labels.append(list(corr_dict[sub].keys())[0])
        if(list(corr_dict[sub].keys())[0] == 0):
            
            diagrams_health.append(rips.fit_transform(1 - np.abs(dist), distance_matrix=True)[dim])
        else:
            diagrams_disease.append(rips.fit_transform(1 - np.abs(dist), distance_matrix=True)[dim])
        #print('sub', sub)
        #hope.write(str(sub)+','+str(list(corr_dict[sub].keys())[0])+'\n')  
        #находим персистетные диаграммы из корреляционной матрицы делая 1 - |corr_matr|
        w = rips.fit_transform(1 - np.abs(dist), distance_matrix=True)[dim]
        diagrams_h1.append(w)
    return diagrams_h1, diagrams_health, diagrams_disease, labels

In [ ]:

dall1, dh, ddis, lb = all_diagr(1, 0.9, data_dir_new , 'aroma', "adhd",1, "1aroma_check_all_SCHZ.txt", "aroma_SCHZ_features.csv", "keys_aroma-schz.txt",2, 2.1, 0.2)

dall2, dh, ddis, lb1 = all_diagr(2, 0.9, data_dir_new , 'aroma', "adhd",1, "1aroma_check_all_SCHZ.txt", "aroma_SCHZ_features.csv", "keys_aroma-schz.txt",2, 2.1, 0.2)


In [17]:


diags_tmp = {'Alpha0': dall1,  # end of the first diagram type
           'Alpha1': dall1
           }

In [14]:
diags_tmp

{'Alpha0': [array([[0.0947942 , 0.0948695 ],
         [0.0808341 , 0.09367484],
         [0.07034904, 0.0716669 ],
         [0.0640179 , 0.0653044 ],
         [0.0583234 , 0.0594491 ],
         [0.0570142 , 0.0605743 ],
         [0.0553174 , 0.06125385],
         [0.05464315, 0.05755067],
         [0.0526479 , 0.0541538 ],
         [0.05090535, 0.0605067 ],
         [0.0496336 , 0.0527784 ],
         [0.0446383 , 0.04479885],
         [0.0408123 , 0.0418695 ],
         [0.0352796 , 0.03589857]]), array([[0.09789866, 1.        ],
         [0.0975369 , 0.0975716 ],
         [0.0904015 , 0.0912099 ],
         [0.08624864, 1.        ],
         [0.08560514, 1.        ],
         [0.0853738 , 0.0861836 ],
         [0.0848481 , 0.0885007 ],
         [0.08440363, 1.        ],
         [0.08195597, 1.        ],
         [0.0818047 , 0.086488  ],
         [0.07836497, 0.0801371 ],
         [0.0761813 , 0.0865876 ],
         [0.07517844, 0.07832336],
         [0.07394964, 0.0868891 ],
         [

In [18]:
diags, filts = preprocess(diags_tmp)

In [19]:
lb = np.asarray(lb).reshape(1,50)

all_ar = []
for i in lb[0] :
    e = np.array([i, 1-i])
    all_ar.append(e)
all_ar = np.array(all_ar)


layer_type = "im"
weight = "grid"
grid_size = [10,10] 
perm_op = "sum"
keep = 5
image_size=[10, 10]
# Parameter specific to layer_type="gs"
num_gaussians=50
# Parameter specific to layer_type="pm"
d = 50  # Output dimension
# Parameter specific to layer_type="ls"
num_samples = 50

perslayParameters = {"layer_type":layer_type,
                     "perm_op": perm_op, "keep":keep,
                     "weight":weight, "grid_size":grid_size,
                     "image_size": image_size,
                     "num_gaussians": num_gaussians,
                     "pm_dimension": d,
                     "num_samples": num_samples}

feats = np.array([[]]*50)

test_size = 0.1
decay = 0.9
lr = 0.01
num_epoch = 100
optim_parameters = {"decay":decay,"lr":lr, "num_epoch":num_epoch}

model = baseModel(perslayParameters, filts, all_ar)

In [16]:
type(diags)


dall1 = np.array(dall1)

In [ ]:
single_run(diags, feats, all_ar, filts, model, optim_parameters, test_size)

Filtrations used:
dict_keys(['Alpha0', 'Alpha1'])
Thresholding in diagrams: 500
 ***** PersLay parameters: *****
Layer type: im
image size: [10, 10]
grid size: [10, 10]
***** Optimization parameters *****
Optimizer: adam
Number of epochs: 100
Learning rate: 0.01
Decay: 0.9
********************
45 train points and 5 test points
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


# ADHD USA dastaset

In [3]:
data_dir0 = '../server/aroma/_task-rest_bold_space-MNI152NLin2009cAsym_preproc/connectivity_matrices/'
# data_dir1 = '../fcla5study/fmriprep/_task-rest_bold_space-MNI152NLin2009cAsym_preproc/connectivity_matrices/'
# data_dir2 = '../fcla5study/openneuro/_task-rest_bold_space-MNI152NLin2009cAsym_preproc/connectivity_matrices/'

data = pd.read_csv('LA5study_targets.csv')
data['participant_id'] = data['participant_id'].apply(lambda x: re.findall(r'\b\d+\b',x)[0])
data["help"] = data["diagnosis"].apply(lambda x: 0 if x == "CONTROL" else 1 if x =='SCHZ' else 2 if x == 'BIPOLAR' else 3)
dic = pd.Series(data.help.values,index=data.participant_id).to_dict()

In [5]:
def all_diagr(dim, thresh, data_dir,hint, disease_name, disease, name_stat, name_features, name_keys, begin, end, step):
    #---------------------------------------------------------------------------------------------
    #data_dir - directory of data
    #disease - label of disease (1- SCH, 2-Bipolar, 3 - ADHD)
    #name_stat file recording all statistics of features
    #name_features - file recording features for the future classification by Katia
    #name_keys - file of labels (keys)
    #begin - start value of the radious of rips complex
    #end - final value of the radious of rips complex
    #step - step of value of the radious of rips complex
    #---------------------------------------------------------------------------------------------
    #initialization of methods for features
    labels = []   
    fs = os.listdir(data_dir)
    fs.sort()
    import re
    corr_dict = {}
    subs = [] #массив номеров поциентов
    diagrams_h1 = []
    diagrams_health = []
    diagrams_disease = []
    i = 0
    diagrams_h1 = [] #массив для персистетных диаграмм всех пациентов
    with open(name_stat, 'w') as file:
        for ind in np.arange(begin, end, step):
            rips = Rips(maxdim=2, coeff=2, do_cocycles=False,thresh=ind);
            for f in fs:
                
                if (f == '.ipynb_checkpoints' or f == '.DS_Store'):
                    continue
                #sub = f[4:8]
                #print('d')
                sub = re.findall(r'\b\d+\b',f)[0] #выделяем в имени файла пациента его порядковый номер
                #print('sub', sub)
                subs.append(sub) 
                test = dic.get(sub) #находим метку болезни по номеру пациента из словаря
                #print('test',test)
                if sub not in corr_dict: #если пациента еще нет в словаре
                    #print(sub)
                  
                    if (test == 0 or test == disease):  #смотрим, относится ли пациент к классу здоровых или выбранной категории больных
                        
                        corr_dict[sub] = {} #заводим словарь {номер пациента - корреляционная матрица}
                        #corr_dict[sub][test] = np.loadtxt(data_dir+f, delimiter=',', skiprows=4)
                        #print("before down")
                        corr_dict[sub][test] = pd.read_csv(data_dir+f, index_col=[0]).values #запись в словарь корелляционной матрицы
                        #print("after down")
                        corr_dict[sub][test][np.abs(corr_dict[sub][test])< thresh] = 0
                        #corr_dict[sub][test][corr_dict[sub][test]< thresh] = 0
                        #corr_dict[sub][test] = corr_dict[sub][test].transpose()
    for i, sub in enumerate(corr_dict):
        dist = np.asanyarray(list(corr_dict[sub].values()))[0]
        labels.append(list(corr_dict[sub].keys())[0])
        if(list(corr_dict[sub].keys())[0] == 0):
            
            diagrams_health.append(rips.fit_transform(1 - np.abs(dist), distance_matrix=True)[dim])
        else:
            diagrams_disease.append(rips.fit_transform(1 - np.abs(dist), distance_matrix=True)[dim])
        #print('sub', sub)
        #hope.write(str(sub)+','+str(list(corr_dict[sub].keys())[0])+'\n')  
        #находим персистетные диаграммы из корреляционной матрицы делая 1 - |corr_matr|
        w = rips.fit_transform(1 - np.abs(dist), distance_matrix=True)[dim]
        diagrams_h1.append(w)
    return diagrams_h1, diagrams_health, diagrams_disease, labels

In [19]:

dall1, dh, ddis, lb = all_diagr(1, 0.9, data_dir0 , 'aroma', "adhd",2, "1aroma_check_all_SCHZ.txt", "aroma_SCHZ_features.csv", "keys_aroma-schz.txt",2, 2.1, 0.2)

#dall2, dh, ddis, lb1 = all_diagr(2, 0.9, data_dir0 , 'aroma', "adhd",2, "1aroma_check_all_SCHZ.txt", "aroma_SCHZ_features.csv", "keys_aroma-schz.txt",2, 2.1, 0.2)



Rips(maxdim=2, thresh=2.0, coeff=2, do_cocycles=False, n_perm = None, verbose=True)


In [20]:
len(dall1)

171

In [12]:
diags_tmp = {'Alpha0': dall1,  # end of the first diagram type
           'Alpha1': dall1
           }
diags, filts = preprocess(diags_tmp)



In [15]:
lb = np.asarray(lb).reshape(1,171)

all_ar = []
for i in lb[0] :
    e = np.array([i, 3-i])
    all_ar.append(e)
all_ar = np.array(all_ar)


layer_type = "im"
weight = "grid"
grid_size = [10,10] 
perm_op = "sum"
keep = 5
image_size=[10, 10]
# Parameter specific to layer_type="gs"
num_gaussians=50
# Parameter specific to layer_type="pm"
d = 50  # Output dimension
# Parameter specific to layer_type="ls"
num_samples = 50

perslayParameters = {"layer_type":layer_type,
                     "perm_op": perm_op, "keep":keep,
                     "weight":weight, "grid_size":grid_size,
                     "image_size": image_size,
                     "num_gaussians": num_gaussians,
                     "pm_dimension": d,
                     "num_samples": num_samples}

feats = np.array([[]]*171)

test_size = 0.1
decay = 0.9
lr = 0.01
num_epoch = 100
optim_parameters = {"decay":decay,"lr":lr, "num_epoch":num_epoch}

model = baseModel(perslayParameters, filts, all_ar)

In [ ]:
single_run(diags, feats, all_ar, filts, model, optim_parameters, test_size)